In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [2]:
#Need to upload the file
from google.colab import files
uploaded = files.upload()

Saving sherlock-holm.es_stories_plain-text_advs.txt to sherlock-holm.es_stories_plain-text_advs.txt


In [3]:
#Mount Drive
from google.colab import drive as mountGoogleDrive
mountGoogleDrive.mount('/content/FYP')

Mounted at /content/FYP


In [4]:
#Read the text file
with open('sherlock-holm.es_stories_plain-text_advs.txt' , 'r', encoding='utf-8') as file:
  text = file.read()

##The following method analyzes the text and builds a vocabulary of unique words, assigning each word a numerical index

In [5]:
#Now the Tokinization
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [6]:
#Now forming the N-grams from the sequences
input_sequences = []
for line in text.split('\n'):
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [9]:
#input sequences are padded to have same length
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
#split into input and output
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [ ]:
#output is converted to one-hot encode vectors
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [ ]:
#building a neural network architecture to train the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
#training the model
model.compile(loss='categorical_crossentropy' , optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 47s 13ms/step - loss: 6.2463 - accuracy: 0.0757
Epoch 2/100
3010/3010 [==============================] - 23s 8ms/step - loss: 5.5162 - accuracy: 0.1244
Epoch 3/100
3010/3010 [==============================] - 23s 8ms/step - loss: 5.1318 - accuracy: 0.1479
Epoch 4/100
3010/3010 [==============================] - 22s 7ms/step - loss: 4.8102 - accuracy: 0.1657
Epoch 5/100
3010/3010 [==============================] - 22s 7ms/step - loss: 4.5133 - accuracy: 0.1823
Epoch 6/100
3010/3010 [==============================] - 22s 7ms/step - loss: 4.2287 - accuracy: 0.2027
Epoch 7/100
3010/3010 [==============================] - 22s 7ms/step - loss: 3.9576 - accuracy: 0.2248
Epoch 8/100
3010/3010 [==============================] - 22s 7ms/step - loss: 3.7019 - accuracy: 0.2548
Epoch 9/100
3010/3010 [==============================] - 21s 7ms/step - loss: 3.4610 - accuracy: 0.2875
Epoch 10/100
3010/3010 [==============================] - 22s 7

In [ ]:
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 18ms/step
I will leave if they were a little


In [ ]:
#save the model
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
seed_text = "I love"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 26ms/step
I ate is john george
